In [1]:
from asap.data.dataloader import DataSet
from asap.util import topics
import numpy as np
import pandas as pd

import matplotlib.pylab as plt
import seaborn as sns
import colorcet as cc

sample_in = '/home/BCCRC.CA/ssubedi/projects/experiments/asapp/data/simdata/simdata_a_1000_r_0.75_d_10000_s_300_sd_10'
sample_out = '/home/BCCRC.CA/ssubedi/projects/experiments/asapp/result/simdata/a_1000_r_0.75_d_10000_s_300_sd_10/'

dl = DataSet(sample_in,sample_out,data_mode='sparse',data_ondisk=False)
dl.initialize_data()
print(dl.inpath)
print(dl.outpath)


/home/BCCRC.CA/ssubedi/projects/experiments/asapp/data/simdata/simdata_a_1000_r_0.75_d_10000_s_300_sd_10
/home/BCCRC.CA/ssubedi/projects/experiments/asapp/result/simdata/a_1000_r_0.75_d_10000_s_300_sd_10/


In [2]:
dl.load_data()
dl.mtx.shape

(4500, 57773)

In [4]:
import anndata
import scanpy as sc

adata = anndata.AnnData(dl.mtx)
dfvars = pd.DataFrame(dl.cols)
dfobs = pd.DataFrame(dl.rows)
adata.obs = dfobs
adata.var = dfvars

sc.pp.filter_cells(adata, min_genes=0)
sc.pp.filter_genes(adata, min_cells=0)
adata.var['mt'] = adata.var_names.str.startswith('MT-')  
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

# sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')

sc.pp.normalize_total(adata, target_sum=10000)

sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
adata = adata[:, adata.var.highly_variable]

sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])
sc.pp.scale(adata, max_value=10)

sc.tl.pca(adata, svd_solver='arpack')
# sc.pl.pca(adata)
# plt.savefig('_scanpy_raw_pipeline_pca.png');plt.close()

# sc.pl.pca_variance_ratio(adata, n_pcs=50,log=True)
# plt.savefig('_scanpy_raw_pipeline_pca_var.png');plt.close()

sc.pp.neighbors(adata)
sc.tl.umap(adata)
sc.tl.leiden(adata)


/home/BCCRC.CA/ssubedi/.conda/envs/ssubedi/lib/python3.9/site-packages/anndata/_core/anndata.py:120: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/home/BCCRC.CA/ssubedi/.conda/envs/ssubedi/lib/python3.9/site-packages/sklearn/manifold/_spectral_embedding.py:260: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


In [11]:
adata

AnnData object with n_obs × n_vars = 4500 × 2044
    obs: 0, 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden'
    var: 0, 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'log1p', 'hvg', 'pca', 'neighbors', 'umap', 'leiden'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'distances', 'connectivities'

In [51]:

from anndata import AnnData
import scanpy as sc
import numpy as np

cols = pd.DataFrame(dl.cols).set_index(0)
rows = pd.DataFrame(dl.rows).set_index(0)
adata = AnnData(dl.mtx,rows,cols)
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)



In [52]:
adata.obs['pmf'] = df_umap['topic_bulk'].values

In [54]:
adata.obs

,pmf
0,
"0_Tcell,CD4,naive[activated]",2
"1_Tcell,CD4,naive[activated]",2
"2_Tcell,CD4,naive[activated]",2
"3_Tcell,CD4,naive[activated]",2
"4_Tcell,CD4,naive[activated]",2
...,...
"295_Tcell,CD8,naive[activated]",7
"296_Tcell,CD8,naive[activated]",7
"297_Tcell,CD8,naive[activated]",7


In [53]:
sc.tl.rank_genes_groups(adata, 'pmf', method='wilcoxon')

AttributeError: Can only use .cat accessor with a 'category' dtype

In [ ]:

dfall = pd.DataFrame()
for group in groups:
    print(group)
    cols = list(adata.uns['rank_genes_groups']['names'][group])
    df = pd.DataFrame(adata.uns['rank_genes_groups']['scores'][group]).T
    df.columns = cols
    dfall = pd.concat([dfall,df],axis=0,ignore_index=True)

In [ ]:
bulk_data = '/home/BCCRC.CA/ssubedi/projects/experiments/asapp/resources/dice/mean_tpm_merged.csv.gz'
df_dice = pd.read_csv(bulk_data,compression='zip')

df_dice = df_dice.set_index('gene').T

In [ ]:
df_beta.index

In [ ]:
sel_top_genes = list(np.unique(topics.get_topic_top_genes(df_beta.iloc[:,:],top_n=10)['Gene'].values))
K=10
corr=[]
for i in df_beta.index:
    cr = []
    for j in df_dice.index:
        cr.append(np.corrcoef(df_beta.loc[i,sel_top_genes],df_dice.loc[j,sel_top_genes])[0,1])
    corr.append(cr)

df_corr = pd.DataFrame(corr)
df_corr.index = df_beta.index
df_corr.columns = df_dice.index
sns.clustermap(df_corr)
plt.xlabel('sc')
plt.ylabel('bulk')

In [ ]:
df_match = df_umap[['cell','cell_type','topic_bulk']]
df_match = df_match.groupby(['cell_type','topic_bulk']).count().reset_index()
df_match = df_match.pivot('cell_type','topic_bulk')
df_match = df_match.fillna(0)
df_match

In [ ]:
sns.clustermap(df_match)

In [ ]:
df_match.values

In [ ]:
from sklearn.metrics import normalized_mutual_info_score

normalized_mutual_info_score(df_umap['cell_type'].values,df_umap['topic_bulk'].values)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled = scaler.fit_transform(df_theta.to_numpy())
kmeans = KMeans(n_clusters=25, init='k-means++',random_state=0).fit(scaled)
df_umap['topic_bulk'] = kmeans.labels_
df_umap['cell_type'] = [x.split('_')[1] for x in df_umap['cell']]